In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import scipy.linalg

from graph_utils import graph_gen, find_alpha_lower_bound
from problem import Synthetic
from plot_utils import MyFigure

from algorithms import IPLUX, UDC, dual_subgradient
# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')


In [ ]:
# generate problem

# network

num_node = 20
num_edge = 40

# np.random.seed(1)

graph_gen(num_node, num_edge, B=1)

net_dir = f'data/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)
# print(network)

# problem data

parameters = {}
parameters['N'] = num_node
parameters['d'] = 3 # dimension of x_i's
parameters['m'] = 1 # number jof inequality constraints
parameters['p'] = 5 # number of equality constraints

prob = Synthetic(parameters)
# prob = Synthetic(parameters, debug=True)

prob.gen()
prob.load()
alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')

Graph with 20 nodes and 180 edges
(20, 20)
generating a Synthetic problem: N=20, d=1, m=1, p=1
Q: (20, 1), P: (20, 1, 1)
A: (20, 1, 1)
a: (20, 1), c: (20,)
aa: (20, 1), cc: (20,)
x* (20, 1), f* -0.14240280607467687
generated problem saved in data/problem/Synthetic/N20

loading a Synthetic problem, N=20
problem loaded:
Q: (20, 1), P: (20, 1, 1)
A: (20, 1, 1)
a: (20, 1), c: (20,)
aa: (20, 1), ca: (20,)
x_star (20, 1)
opt_val -0.14240280607467687
x* [[ 1.00536999e-11]
 [-2.71219259e-11]
 [ 2.30557228e-09]
 [-2.53286588e-11]
 [ 6.42949940e-11]
 [-3.18451352e-11]
 [ 1.77968755e-10]
 [-2.06218139e-01]
 [ 2.93491319e-01]
 [ 3.01519167e-11]
 [ 2.52431278e-01]
 [-1.80830853e-11]
 [-1.19593210e-01]
 [ 1.61341128e-10]
 [ 2.42764564e-11]
 [ 1.09140187e-01]
 [-4.40031288e-11]
 [-3.72710374e-11]
 [-3.15817259e-02]
 [-2.97669711e-01]]
||x*|| 0.5551607009526415



In [3]:
# load instance 
instance_name = 'instance1_N20E40'
# instance_name = 'instance2_N20E26'
# instance_name = 'instance3_N50E70_identical_cons'
# instance_name = 'instance4_N20E150'
# instance_name = 'instance5_N20E170'
instance_dir = 'instance/' + instance_name

# network
network = np.load(instance_dir + '/graph/subgraph_W.npy')
num_node = network.shape[0]
print(network.shape)

# problem data
parameters = {}
parameters['N'] = num_node
parameters['d'] = 1 # temporary, reset in load()
parameters['m'] = 1 # temporary
parameters['p'] = 1 # temporary

prob = Synthetic(parameters)
prob.save_dir = instance_dir + '/problem'
prob.load()

alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')


(20, 20)
loading a Synthetic problem, N=20
problem loaded:
Q: (20, 3), P: (20, 3, 3)
A: (20, 5, 3)
a: (20, 3), c: (20,)
aa: (20, 3), ca: (20,)
x_star (20, 3)
opt_val -1.7937811849951981
x* [[ 1.69192667e-08 -4.54451916e-02 -1.25326407e-02]
 [-1.97438972e-11  2.94708451e-10  1.96457124e-01]
 [-1.55498077e-01 -1.81713964e-10 -4.30739243e-02]
 [ 2.21417334e-10  6.41377197e-11 -2.57323583e-01]
 [ 5.46491556e-01 -3.09078549e-01 -2.25841238e-10]
 [-7.95790832e-10  4.77051274e-01 -9.77296145e-02]
 [ 2.30311170e-01  9.67073127e-10 -4.08278630e-02]
 [ 4.41710887e-10  5.03547130e-01  2.55567666e-02]
 [-4.78912681e-01 -1.55547314e-09  3.47456666e-10]
 [ 1.16595532e-10 -9.51339774e-11 -4.60428545e-01]
 [ 4.42020248e-10 -4.19241406e-01  1.76861413e-01]
 [-1.46017592e-09  6.03865377e-10 -1.04286398e-09]
 [-3.70053317e-01  6.45150319e-01  3.92380391e-10]
 [ 1.95357925e-10 -2.86013131e-10 -4.44910108e-10]
 [-1.46178792e-08 -3.47711897e-01  8.41473423e-10]
 [-8.31629811e-10 -1.66205909e-01  1.02254197e

In [ ]:
# IPLUX
# Instance 1: a5.0 r5
# Instance 2: a20_r4 130.59s

MAX_ITER = 2000
# MAX_ITER = 500

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

alpha_lower = find_alpha_lower_bound(parameters, prob)
# alpha_choice = [alpha_lower, alpha_lower*5, alpha_lower*25]
alpha_choice = [alpha_lower, alpha_lower+1]
# alpha_choice = [alpha_lower+1]

# rho_choice = [1, 5, 25, 0.5]
rho_choice = [0.5, 1, 5, 10]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')

np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        # if True:
        # if rho != 5:
        if rho == 10 and alpha == 5.0:
            alg = IPLUX(prob, network, alpha=alpha, rho=rho, verbose=False)
            for i in range(MAX_ITER):
                alg.step()
        
        prefix = f'{alg.name}_a{alpha}_r{rho}'
        filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
        filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
        filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
        
        obj_err_figure.add_line_file(prefix, filename_oe, style=c+ls)
        cons_vio_figure.add_line_file(prefix, filename_cv, style=c+ls)
        x_dis_figure.add_line_file(prefix, filename_xd, style=c+ls)
            
        obj_err_figure.paint(MAX_ITER=MAX_ITER)
        cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
        x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

In [13]:
alg = UDC(prob, network, rho=1, alpha=1,
                    param_setting='New2', verbose=False)

UDC setting: New2
[[0.9047619  0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         1.48809524 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.53333333 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         1.45       0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.     

In [ ]:
# UDC

# best parameters:

# instance 1
# UDC_PEXTRA: r4
# UDC_PGC: r0.4
# UDC_DPGA: r2
# UDC_DistADMM: r1
# ALT: r4
# New1: r2
# New2: r2

# instance 2
# UDC_PEXTRA: a0_r1
# UDC_pt: a20_r4, 156.29s
# UDC_DistADMM: a0_r2

# instance 3
# UDC_PEXTRA: r2
# UDC_PGC: r0.4
# UDC_DPGA: r2
# UDC_DistADMM: r1
# ALT: r2

# instance 4
# UDC_PEXTRA: r
# UDC_PGC: r
# UDC_DPGA: r
# UDC_DistADMM: 
# ALT: r2
# New1: r1

# instance 5
# UDC_PEXTRA: r
# UDC_PGC: r
# UDC_DPGA: r
# UDC_DistADMM: 
# ALT: r3
# New1: r1

MAX_ITER = 500
# MAX_ITER = 1000
# MAX_ITER = 2000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

rho_choice = [0.1, 1, 10]
# rho_choice = [0.5, 1, 2, 3, 4]
# rho_choice = [0.05, 0.1, 0.2, 0.4, 0.8]
# rho_choice = [2, 2.5]
# rho_choice = [2, 4]
# rho_choice = [2]

# alpha_choice = [0, 10, 100]
# alpha_choice = [0.1, 0.2, 0.5]
# alpha_choice = [0, 0.1, 0.5]
alpha_choice = [0]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')
    
np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        if True:
        # if rho == 0.5:
        # if rho != 1:
        # if rho == 3:
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #           param_setting='PEXTRA', verbose=False)
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #           param_setting='PGC', verbose=False)
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #           param_setting='DPGA', verbose=False)
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #           param_setting='DistADMM', verbose=False)
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #         param_setting='ALT', verbose=False)
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #         param_setting='New1', verbose=False)
            alg = UDC(prob, network, rho=rho, alpha=alpha,
                    param_setting='New2', verbose=False)
            for i in range(MAX_ITER):
                alg.step()
            
        prefix = f'{alg.name}_a{alpha}_r{rho}'
        filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
        filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
        filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
        
        obj_err_figure.add_line_file(prefix, filename_oe, style=c+ls)
        cons_vio_figure.add_line_file(prefix, filename_cv, style=c+ls)
        x_dis_figure.add_line_file(prefix, filename_xd, style=c+ls)
            
        obj_err_figure.paint(MAX_ITER=MAX_ITER)
        cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
        x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


In [ ]:
# DPMM
# instance 1: a0.1_r4_t1.5
# instance 2: a0.1_r1_t1.5

from plot_utils import MyFigure

alg_name = 'DPMM'
MAX_ITER = 2000
# MAX_ITER = 500

# theta_choice = [0.5, 1, 1.5, 1.9]
# theta_choice = [0.1, 1.9]
theta_choice = [1.5]

# rho_choice = [0.1, 1, 10]
# rho_choice = [0.5, 1, 2, 4, 5]
rho_choice = [4]

# alpha_choice = [0, 10, 100]
# alpha_choice = [0.1, 1, 10]
alpha_choice = [0.1]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')


for alpha in alpha_choice:
    for rho in rho_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        for theta in theta_choice:
            
            if True:
            # if rho != 1:
            # if rho == 5:
            # if alpha != 0:
            # if theta == 1.9:
                alg = UDC(prob, network, rho=rho, alpha=alpha, theta=theta,
                            param_setting='DPMM', verbose=False)
                for i in range(MAX_ITER):
                    alg.step()

            # plot
            # ci = theta_choice.index(theta)
            # c = color[ci]
            
            prefix = f'{alg_name}_a{alpha}_r{rho}_t{theta}'
            filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
            filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
            filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
            
            obj_err_figure.add_line_file(prefix, filename_oe, style=c+ls)
            cons_vio_figure.add_line_file(prefix, filename_cv, style=c+ls)
            x_dis_figure.add_line_file(prefix, filename_xd, style=c+ls)
                
            obj_err_figure.paint(MAX_ITER=MAX_ITER)
            cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
            x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


In [ ]:
# dual subgradient
#
# instance 1: g5
# instance 2: g10

from plot_utils import MyFigure

alg_name = 'dual_subgradient'
MAX_ITER = 500
# gamma_choice = [0.1, 1, 5, 25]
# gamma_choice = [2.5, 5, 10, 50]
gamma_choice = [2.5, 5, 10]
color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')


for gamma in gamma_choice:
    if True:
    # if gamma == 50:
        alg = dual_subgradient(prob, network, gamma)
        for i in range(MAX_ITER):
            alg.step()

    # plot
    ci = gamma_choice.index(gamma)
    c = color[ci]
    
    prefix = f'{alg_name}_g{gamma}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    obj_err_figure.add_line_file(prefix, filename_oe, style=c)
    cons_vio_figure.add_line_file(prefix, filename_cv, style=c)
    x_dis_figure.add_line_file(prefix, filename_xd, style=c)
        
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

    

In [4]:
# compare fine-tuned algorithms: load data

# instance 1
# UDC_PEXTRA: r4
# UDC_PGC: r0.4
# UDC_DPGA: r2
# UDC_DistADMM: r1
# UDC_New1: r2
# ALT: r4
# DPMM: a0.1 r4 t1.5
# IPLUX: a5.0 r5
# dual subgradient: g5

# instance 4
# UDC_PEXTRA: r
# UDC_PGC: r
# UDC_DPGA: r
# UDC_DistADMM: r
# UDC_New1: r1
# ALT: r2
# DPMM: a r t
# IPLUX: a r
# dual subgradient: g

# instance 5
# UDC_PEXTRA: r
# UDC_PGC: r
# UDC_DPGA: r
# UDC_DistADMM: r
# UDC_New1: r1
# ALT: r3
# DPMM: a r t
# IPLUX: a r
# dual subgradient: g


alpha_list = [0, 0.1, 0.5]

alg_list = []
for a in alpha_list:
    alg_list.append(UDC(prob, network, alpha=a, rho=4, param_setting='PEXTRA'))
    alg_list.append(UDC(prob, network, alpha=a, rho=0.4, param_setting='PGC'))
    alg_list.append(UDC(prob, network, alpha=a, rho=2, param_setting='DPGA'))
    alg_list.append(UDC(prob, network, alpha=a, rho=1, param_setting='DistADMM'))
    alg_list.append(UDC(prob, network, alpha=a, rho=2, param_setting='New1'))
    alg_list.append(UDC(prob, network, alpha=a, rho=4, param_setting='ALT'))
    
alg_list.append(UDC(prob, network, alpha=0.1, rho=4, theta=1.5, 
                                                param_setting='DPMM'))
alg_list.append(IPLUX(prob, network, alpha=5.0, rho=5))
alg_list.append(dual_subgradient(prob, network, gamma=5))

oe_logs = dict()
cv_logs = dict()
xd_logs = dict()

UDC_list = ['UDC_PEXTRA', 'UDC_PGC', 'UDC_DPGA', 'UDC_DistADMM', 'UDC_New1', 'ALT']
# UDC_list = ['UDC_PEXTRA']
for u in UDC_list:
    oe_logs[u] = dict()
    cv_logs[u] = dict()
    xd_logs[u] = dict()


# instance 2
# param_best_list = [['IPLUX', 4.0, 1], 
#                  ['UDC_pt', 20, 4], 
#                  ['UDC_PEXTRA', 0.1, 1], 
#                  ['UDC_DistADMM', 0.1, 2],
#                  ['DPMM', 0.1, 1, 1.5],
#                  ['dual_subgradient', 10]]
# alg_list = []
# alg_list.append(IPLUX(prob, network, alpha=4.0, rho=1))
# alg_list.append(UDC(prob, network, alpha=20, rho=4, param_setting='pt'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=1, param_setting='PEXTRA'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=2, param_setting='DistADMM'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=1, theta=1.5, param_setting='DPMM'))
# alg_list.append(dual_subgradient(prob, network, gamma=10))




for alg in alg_list:
    
    # instance 1
    log_dir = f'instance/instance1_N20E40/log'
    # instance 4
    # log_dir = f'instance/instance4_N20E150/log'
    # instance 5
    # log_dir = f'instance/instance5_N20E170/log'
    file_alg_name = alg.name.replace(' ', '_').replace('-', '_')
    filename_oe = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_oe.txt'
    filename_cv = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_cv.txt'
    filename_xd = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_xd.txt'
    
    if file_alg_name[:3] == 'UDC' or file_alg_name[:3] == 'ALT':
    # if file_alg_name == 'UDC_PEXTRA':
        oe_logs[file_alg_name][alg.alpha] = np.loadtxt(filename_oe)
        cv_logs[file_alg_name][alg.alpha] = np.loadtxt(filename_cv)
        xd_logs[file_alg_name][alg.alpha] = np.loadtxt(filename_xd)
    else:
        oe_logs[file_alg_name] = np.loadtxt(filename_oe)
        cv_logs[file_alg_name] = np.loadtxt(filename_cv)
        xd_logs[file_alg_name] = np.loadtxt(filename_xd)
    


UDC setting: PEXTRA
self.prob minimize quad_over_lin(param1042 @ var1041, 1.0) + param1043 @ var1041 + param1044[0] @ norm1(var1041) + 0.0 @ quad_over_lin(Promote(param1045[0], (3,)) @ var1041 + -param1046, 1.0) + 0.5 @ quad_over_lin(maximum(param1047 + quad_over_lin(var1041 + -param1049, 1.0) + -param1050, 0.0), 1.0) + 0.5 @ quad_over_lin(param1048 + param1053 @ var1041, 1.0)
subject to QuadForm(var1041 + -param1051, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param1052[0]
reset
UDC_PEXTRA alpha 0 rho 4, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.01, saved

UDC setting: PGC
self.prob minimize quad_over_lin(param1094 @ var1093, 1.0) + param1095 @ var1093 + param1096[0] @ norm1(var1093) + 0.0 @ quad_over_lin(Promote(param1097[0], (3,)) @ var1093 + -param1098, 1.0) + 0.5 @ quad_over_lin(maximum(param1099 + quad_over_lin(var1093 + -param1101, 1.0) + -param1102, 0.0), 1.0) + 0.5 @ quad_over_lin(param1100 + param1105 @ var1093, 1.0)
subject to QuadForm(var1093 + -pa

In [24]:

from scipy.linalg import eigvalsh
from math import sqrt

test_list = []
# test_list.append(UDC(prob, network, alpha=0, rho=4, param_setting='PEXTRA'))
# test_list.append(UDC(prob, network, alpha=0, rho=0.4, param_setting='PGC'))
# test_list.append(UDC(prob, network, alpha=0, rho=2, param_setting='DPGA'))
# test_list.append(UDC(prob, network, alpha=0, rho=4, param_setting='DistADMM'))
# test_list.append(UDC(prob, network, alpha=0, rho=1, param_setting='ALT'))
# test_list.append(UDC(prob, network, alpha=0, rho=2, param_setting='ALT'))
# test_list.append(UDC(prob, network, alpha=0, rho=4, param_setting='ALT'))
# test_list.append(UDC(prob, network, alpha=0, rho=1, param_setting='New1'))
test_list.append(UDC(prob, network, alpha=0, rho=2, param_setting='New1'))
test_list.append(UDC(prob, network, alpha=0, rho=2, param_setting='New2'))
# test_list.append(UDC(prob, network, alpha=0, rho=1, param_setting='ALT'))
# test_list.append(UDC(prob, network, alpha=0, rho=2, param_setting='ALT'))
# test_list.append(UDC(prob, network, alpha=0, rho=3, param_setting='ALT'))
# test_list.append(UDC(prob, network, alpha=0, rho=4, param_setting='ALT'))

N = num_node
I = np.identity(N)
Mg = I - network
W = network
ones = np.ones((N,1))
print(f'W {eigvalsh(W)}')
print(f'W-1/N 11^T {eigvalsh(W-ones@ones.T/N)}')
print(f'I-W^2 - (I-W)^2 {eigvalsh(I-W@W - Mg@Mg)}')
# print(f'Mg {eigvalsh(Mg)}')
# print(f'I-Mg {eigvalsh(network)}')
# print(f'Mg(I-Mg) {eigvalsh(Mg @ network)}')
print()
for alg in test_list:
    if alg.name[:3] == 'UDC' or alg.name[:3] == 'ALT':
        print(f'{alg.name} r{alg.rho}')
        A, H1, H2, D = alg.A_weight, alg.H1, alg.H2, alg.D
        rho = 1.0 if alg.rho_only_in_mat else alg.rho
        A_max = eigvalsh(A)[-1]
        tilH_min2 = eigvalsh(H2)[1]
        # print(f'A_max: {A_max}')
        # print(f'tilH eigen min2: {tilH_min2}')
        # print(f'rho: {rho}')
        # print(f'rho * tilH_min2: {rho*eigvalsh(H2)[1]}')
        print(f'coeff in OE: {1/(rho*tilH_min2)}||g*||^2')
        print(f'coeff in FE1: {2*N*A_max}||y*|| + {1/(rho*tilH_min2)}||g*||')
        print(f'coeff in FE2: {N*A_max}||y*|| + {sqrt(N*A_max/(rho*tilH_min2))}||g*|| + {0.5*sqrt(N*A_max)}')
        print()
        

UDC setting: New1
self.prob minimize quad_over_lin(param5427 @ var5426, 1.0) + param5428 @ var5426 + param5429[0] @ norm1(var5426) + 0.0 @ quad_over_lin(Promote(param5430[0], (3,)) @ var5426 + -param5431, 1.0) + 0.5 @ quad_over_lin(maximum(param5432 + quad_over_lin(var5426 + -param5434, 1.0) + -param5435, 0.0), 1.0) + 0.5 @ quad_over_lin(param5433 + param5438 @ var5426, 1.0)
subject to QuadForm(var5426 + -param5436, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param5437[0]
reset
UDC_New1 alpha 0 rho 2, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.01, saved

UDC setting: New2
self.prob minimize quad_over_lin(param5479 @ var5478, 1.0) + param5480 @ var5478 + param5481[0] @ norm1(var5478) + 0.0 @ quad_over_lin(Promote(param5482[0], (3,)) @ var5478 + -param5483, 1.0) + 0.5 @ quad_over_lin(maximum(param5484 + quad_over_lin(var5478 + -param5486, 1.0) + -param5487, 0.0), 1.0) + 0.5 @ quad_over_lin(param5485 + param5490 @ var5478, 1.0)
subject to QuadForm(var5478 + -param

In [101]:
oe_logs['UDC_New1'][0]

array( 1.79e+00)

In [81]:

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
from matplotlib.ticker import FormatStrFormatter, LogFormatter
from matplotlib.legend_handler import HandlerTuple
# rc('mathtext', fontset='stixsans')
# rc('text', usetex=True)
# rc('text', usetex=True)
# rc('mathtext', fontset ='dejavusans')
# rc('font', **{'family': 'sans-serif', 'sans-serif': ['DejaVu Sans'], 'size': 40})

MAX_ITER = 1000

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": "Times",
    "font.sans-serif": "Helvetica",
    "font.size": 45,
    # 'text.latex.preamble': r'\usepackage{sfmath}' # sans-serif for math
    'text.latex.preamble': r'\usepackage{newtxtext} \usepackage{amssymb} \usepackage{amsmath}',
    'axes.facecolor': '#e8e8e8'
})

# set line colors
c = {
    'UDC_PEXTRA'        : 'crimson',
    'UDC_PGC'           : '#00f9ff',   # sky blue
    'UDC_DPGA'          : 'goldenrod',   # blue gray
    'UDC_DistADMM'      : 'xkcd:green',
    'ALT'               : 'xkcd:orange',
    'DPMM'              : 'lightseagreen',
    'IPLUX'             : 'xkcd:purple',
    'dual_subgradient'  : 'olive',
    'UDC_New1'          : 'black'
}

# set transparent
al = {
    'UDC_PEXTRA'        : 1.0,
    'UDC_PGC'           : 1.0,
    'UDC_DPGA'          : 1.0,
    'UDC_DistADMM'      : 1.0,
    'ALT'               : 1.0,
    'DPMM'              : 1.0,
    'IPLUX'             : 1.0,
    'dual_subgradient'  : 1.0,
    'UDC_New1'          : 1.0
}

fig, axs = plt.subplot_mosaic([['oe', 'oep'],['cv', 'cvp']], 
                              layout='constrained',
                              figsize=(32, 24))
# fig, ax = plt.subplots(figsize=(20, 12), layout='constrained')   
axs['oe'].set_xlim(0, MAX_ITER)
axs['oe'].set_ylim(1e-9, 9e0)
axs['oe'].set_yscale('log')
axs['oe'].set_xlabel(r'''$\mathrm{(a)}$''', fontsize=45)
# axs['oe'].set_ylabel(r'$|f({\mathbf x}^k)-f({\mathbf x}^\star)|$', fontsize=45)
axs['oe'].set_ylabel(r'objective error', fontsize=45)
axs['oe'].tick_params(axis='both', labelsize=45)
axs['oe'].yaxis.set_minor_locator(plt.LogLocator())
axs['oe'].spines.bottom.set_linewidth(0)
axs['oe'].spines.top.set_linewidth(0)
axs['oe'].spines.left.set_linewidth(0)
axs['oe'].spines.right.set_linewidth(0)
axs['oe'].grid(True, lw=5, color='1')


p4, = axs['oe'].plot(oe_logs['UDC_New1'][0],  
                     label='DUCA-I', 
                     c=c['UDC_New1'],
                     alpha=al['UDC_New1'],
                     lw=5, 
                     ls='-'
                     )
p1, = axs['oe'].plot(oe_logs['UDC_PEXTRA'][0], 
                     label='DUCA-PEXTRA', 
                     c=c['UDC_PEXTRA'],
                     alpha=al['UDC_PEXTRA'],
                     lw=5, 
                     ls='-'
                     )
p2, = axs['oe'].plot(oe_logs['UDC_PGC'][0], 
                     label='DUCA-PGC', 
                     c=c['UDC_PGC'],
                     alpha=al['UDC_PGC'],
                     lw=5, 
                     ls='-'
                     )
p3, = axs['oe'].plot(oe_logs['UDC_DPGA'][0], 
                     label='DUCA-DPGA', 
                     c=c['UDC_DPGA'],
                     alpha=al['UDC_DPGA'],
                     lw=5,
                     ls='-'
                     ) 
p4, = axs['oe'].plot(oe_logs['UDC_DistADMM'][0],  
                     label='DUCA-dist.ADMM', 
                     c=c['UDC_DistADMM'],
                     alpha=al['UDC_DistADMM'],
                     lw=5, 
                     ls='-'
                     )
p5, = axs['oe'].plot(oe_logs['ALT'][0], 
                     label='ALT', 
                     c=c['ALT'],
                     alpha=al['ALT'],
                     lw=5, 
                     ls='-'
                     )
p6, = axs['oe'].plot(oe_logs['DPMM'], 
                     label='DPMM', 
                     c=c['DPMM'],
                     alpha=al['DPMM'],
                    #  marker='*',
                    #  markevery=30,
                    #  markersize=20,
                     lw=5, 
                     ls='-.'
                     )
p7, = axs['oe'].plot(oe_logs['IPLUX'], 
                     label='IPLUX', 
                     c=c['IPLUX'],
                     alpha=al['IPLUX'],
                     lw=5, 
                     ls='-.'
                     )
p8, = axs['oe'].plot(oe_logs['dual_subgradient'], 
                     label='dual subgradient',  
                     c=c['dual_subgradient'], 
                     alpha=al['dual_subgradient'],
                     lw=5,
                     ls='-.'
                     )

# ax.legend([(p1, p2, p3), p4, p5, p6, p7, p8], 
#           ['UDC-SingleRound', 'UDC-Dist.ADMM', 'ALT', 'DPMM', 
#            'IPLUX', 'dual subgradient'], 
#             handler_map={tuple: HandlerTuple(ndivide=None)},
#             fontsize=40)
# fig.legend(fontsize=40, loc='outside upper center', ncols=2)
# bbox (x, y, width, height)
# ax.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
#                       ncols=2, mode="expand", borderaxespad=0.)
# fig.savefig(f'compare_obj_err.pdf', bbox_inches='tight', transparent=False)


# ======================== constraint violation ===========================

# fig, ax = plt.subplots(figsize=(20, 12))   
axs['cv'].set_xlim(0, MAX_ITER)
axs['cv'].set_yscale('log')
axs['cv'].set_ylim(1e-10, 5e1)
axs['cv'].set_xlabel(r'''iteration $k$
                      
                      $\mathrm{(b)}$''', fontsize=45)
axs['cv'].set_ylabel(r'constraint violation', fontsize=45)
axs['cv'].tick_params(axis='both', labelsize=45)
axs['cv'].yaxis.set_minor_locator(plt.LogLocator())
axs['cv'].spines.bottom.set_linewidth(0)
axs['cv'].spines.top.set_linewidth(0)
axs['cv'].spines.left.set_linewidth(0)
axs['cv'].spines.right.set_linewidth(0)
axs['cv'].grid(True, lw=5, color='1')


p4, = axs['cv'].plot(cv_logs['UDC_New1'][0],  
                    #  label='UDC-New1', 
                     c=c['UDC_New1'],
                     alpha=al['UDC_New1'],
                     lw=5, 
                     ls='-'
                     )
p1, = axs['cv'].plot(cv_logs['UDC_PEXTRA'][0], 
                    #  label='UDC-PEXTRA', 
                     c=c['UDC_PEXTRA'],
                     alpha=al['UDC_PEXTRA'],
                     lw=5, 
                     ls='-'
                     )
p2, = axs['cv'].plot(cv_logs['UDC_PGC'][0], 
                    #  label='UDC-PGC', 
                     c=c['UDC_PGC'],
                     alpha=al['UDC_PGC'],
                     lw=5, 
                     ls='-'
                     )
p3, = axs['cv'].plot(cv_logs['UDC_DPGA'][0], 
                    #  label='UDC-DPGA', 
                     c=c['UDC_DPGA'],
                     alpha=al['UDC_DPGA'],
                     lw=5,
                     ls='-'
                     ) 
p4, = axs['cv'].plot(cv_logs['UDC_DistADMM'][0],  
                    #  label='UDC-dist.ADMM', 
                     c=c['UDC_DistADMM'],
                     alpha=al['UDC_DistADMM'],
                     lw=5, 
                     ls='-'
                     )
p5, = axs['cv'].plot(cv_logs['ALT'][0], 
                    #  label='ALT', 
                     c=c['ALT'],
                     alpha=al['ALT'],
                     lw=5, 
                     ls='-'
                     )
p6, = axs['cv'].plot(cv_logs['DPMM'], 
                    #  label='DPMM', 
                     c=c['DPMM'],
                     alpha=al['DPMM'],
                     lw=5, 
                     ls='-.'
                     )
p7, = axs['cv'].plot(cv_logs['IPLUX'], 
                    #  label='IPLUX', 
                     c=c['IPLUX'],
                     alpha=al['IPLUX'],
                     lw=5, 
                     ls='-.'
                     )
p8, = axs['cv'].plot(cv_logs['dual_subgradient'], 
                    #  label='dual subgradient',  
                     c=c['dual_subgradient'], 
                     alpha=al['dual_subgradient'],
                     lw=5,
                     ls='-.'
                     )

# ax.legend([(p1, p2, p3), p4, p5, p6, p7, p8], 
#           ['UDC-SingleRound', 'UDC-Dist.ADMM', 'ALT', 'DPMM', 
#            'IPLUX', 'dual subgradient'], 
#             handler_map={tuple: HandlerTuple(ndivide=None)},
#             fontsize=40)
# fig.savefig(f'compare_cons_vio.pdf', bbox_inches='tight', transparent=False)
first_legend = fig.legend(fontsize=40, bbox_to_anchor=(1, 0.57), loc='upper right', ncols=1, framealpha=0.4)
# bbox (x, y, width, height)
# fig.savefig(f'compare_obj_err.pdf', bbox_inches='tight', transparent=False)



# ================================= Proximal ====================================

fig.add_artist(first_legend)

alpha_list = [0, 0.1, 0.5]
# set linestyle
ls = {0:':', 0.1:'-', 0.5: '-'}
# set transparent
al = {0:1, 0.1:1, 0.5:0.5}
marker = {0:'', 0.1:'', 0.5:'*'}
handles = [] # line handles, for legend


axs['oep'].set_xlim(0, MAX_ITER)
axs['oep'].set_yscale('log')
axs['oep'].set_ylim(1e-9, 9e0)
axs['oep'].set_xlabel(r'$\mathrm{(c)}$', fontsize=45)
# axs['oep'].set_ylabel(r'objective error', fontsize=45)
axs['oep'].spines.bottom.set_linewidth(0)
axs['oep'].spines.top.set_linewidth(0)
axs['oep'].spines.left.set_linewidth(0)
axs['oep'].spines.right.set_linewidth(0)
axs['oep'].grid(True, lw=5, color='1')

for alpha in alpha_list:
    pad = ''
    p0, = axs['oep'].plot(oe_logs['UDC_New1'][alpha],  
                     label=pad+f'$\\alpha={alpha}$', 
                     c=c['UDC_New1'],
                     alpha=al[alpha],
                     lw=5, 
                    #  marker=marker[alpha],
                    #  markevery=50,
                    #  markersize=20,
                     ls=ls[alpha]
                     )
    p1, = axs['oep'].plot(oe_logs['UDC_PEXTRA'][alpha], 
                            label=pad+f'$\\alpha={alpha}$', 
                            c=c['UDC_PEXTRA'],
                            alpha=al[alpha],
                            lw=5, 
                            # marker=marker[alpha],
                            # markevery=50,
                            # markersize=20,
                            # markeredgewidth=3,
                            ls=ls[alpha]
                            )
    # p2, = axs['oep'].plot(oe_logs['UDC_PGC'][alpha], 
    #                         label=pad+f'$\\alpha={alpha}$', 
    #                         c=c['UDC_PGC'],
    #                         alpha=al[alpha],
    #                         lw=5, 
    #                         marker=marker[alpha],
    #                         markevery=50,
    #                         markersize=20,
    #                         ls=ls[alpha]
    #                         )
    # p3, = axs['oep'].plot(oe_logs['UDC_DPGA'][alpha], 
    #                         label=pad+f'$\\alpha={alpha}$', 
    #                         c=c['UDC_DPGA'],
    #                         alpha=al[alpha],
    #                         lw=5, 
    #                         ls=ls[alpha]
    #                         )
    p4, = axs['oep'].plot(oe_logs['UDC_DistADMM'][alpha], 
                            label=pad+f'$\\alpha={alpha}$', 
                            c=c['UDC_DistADMM'],
                            alpha=al[alpha],
                            lw=5, 
                            # marker=marker[alpha],
                            # markevery=50,
                            # markersize=20,
                            ls=ls[alpha]
                            )
    p5, = axs['oep'].plot(oe_logs['ALT'][alpha], 
                            label=pad+f'$\\alpha={alpha}$', 
                            c=c['ALT'],
                            alpha=al[alpha],
                            lw=5, 
                            # marker=marker[alpha],
                            # markevery=50,
                            # markersize=20,
                            ls=ls[alpha]
                            )
    handles += [p0, p2, p4, p5]
handles = handles[::4]

axs['cvp'].set_xlim(0, MAX_ITER)
axs['cvp'].set_yscale('log')
axs['cvp'].set_ylim(1e-10, 5e1)
axs['cvp'].set_xlabel(r'''iteration $k$
                      
                      $\mathrm{(d)}$''', fontsize=45)
# axs['cvp'].set_ylabel(r'objective error', fontsize=45)
axs['cvp'].spines.bottom.set_linewidth(0)
axs['cvp'].spines.top.set_linewidth(0)
axs['cvp'].spines.left.set_linewidth(0)
axs['cvp'].spines.right.set_linewidth(0)
axs['cvp'].grid(True, lw=5, color='1')

for alpha in alpha_list:
    p0, = axs['cvp'].plot(cv_logs['UDC_New1'][alpha],  
                    #  label=f'$\\alpha={alpha}$', 
                     c=c['UDC_New1'],
                     alpha=al[alpha],
                     lw=5, 
                    #  marker=marker[alpha],
                    #         markevery=50,
                    #         markersize=20,
                     ls=ls[alpha]
                     )
    p1, = axs['cvp'].plot(cv_logs['UDC_PEXTRA'][alpha], 
                            # label='UDC-PEXTRA', 
                            c=c['UDC_PEXTRA'],
                            alpha=al[alpha],
                            lw=5, 
                            # marker=marker[alpha],
                            # markevery=50,
                            # markersize=20,
                            ls=ls[alpha]
                            )
    # p1, = axs['cvp'].plot(cv_logs['UDC_PGC'][alpha], 
    #                         # label='UDC-PGC', 
    #                         c=c['UDC_PGC'],
    #                         alpha=al[alpha],
    #                         lw=5, 
    #                         ls=ls[alpha]
    #                         )
    # p1, = axs['cvp'].plot(cv_logs['UDC_DPGA'][alpha], 
    #                         # label='UDC-DPGA', 
    #                         c=c['UDC_DPGA'],
    #                         alpha=al[alpha],
    #                         lw=5, 
    #                         ls=ls[alpha]
    #                         )
    p1, = axs['cvp'].plot(cv_logs['UDC_DistADMM'][alpha], 
                            # label='UDC-dist.ADMM', 
                            c=c['UDC_DistADMM'],
                            alpha=al[alpha],
                            lw=5, 
                            # marker=marker[alpha],
                            # markevery=50,
                            # markersize=20,
                            ls=ls[alpha]
                            )
    p1, = axs['cvp'].plot(cv_logs['ALT'][alpha], 
                            # label='ALT', 
                            c=c['ALT'],
                            alpha=al[alpha],
                            lw=5, 
                            # marker=marker[alpha],
                            # markevery=50,
                            # markersize=20,
                            ls=ls[alpha]
                            )



fig.legend(handles=handles,fontsize=40, loc='upper right', ncols=3)
fig.savefig(f'compare_obj_err.pdf', bbox_inches='tight', transparent=False)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
from matplotlib.ticker import FormatStrFormatter, LogFormatter
from matplotlib.legend_handler import HandlerTuple
# rc('mathtext', fontset='stixsans')
# rc('text', usetex=True)
# rc('text', usetex=True)
# rc('mathtext', fontset ='dejavusans')
# rc('font', **{'family': 'sans-serif', 'sans-serif': ['DejaVu Sans'], 'size': 40})

MAX_ITER = 2000

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.serif": "Computer Modern Roman",
    "font.sans-serif": "Helvetica",
    "font.size": 42,
    # 'text.latex.preamble': r'\usepackage{sfmath}' # sans-serif for math
    'text.latex.preamble': r'\usepackage{amssymb} \usepackage{amsmath}',
    'axes.facecolor': '#e8e8e8'
})

# set line colors
c = {
    'UDC_PEXTRA'        : 'xkcd:blue',
    'UDC_PGC'           : '#00f9ff',   # sky blue
    'UDC_DPGA'          : 'xkcd:olive',   # blue gray
    'UDC_DistADMM'      : 'xkcd:green',
    'ALT'               : 'orange',
    'DPMM'              : 'xkcd:orangered',
    'IPLUX'             : 'xkcd:purple',
    'dual_subgradient'  : 'olive',
    'UDC_New1'          : 'black'
}

# set transparent
al = {
    'UDC_PEXTRA'        : 1.0,
    'UDC_PGC'           : 1.0,
    'UDC_DPGA'          : 1.0,
    'UDC_DistADMM'      : 1.0,
    'ALT'               : 1.0,
    'DPMM'              : 1.0,
    'IPLUX'             : 1.0,
    'dual_subgradient'  : 1.0,
    'UDC_New1'          : 1.0
}

fig, axs = plt.subplot_mosaic([['oe', 'oep'],['cv', 'cvp']], 
                              layout='constrained',
                              figsize=(30, 24))
# fig, ax = plt.subplots(figsize=(20, 12), layout='constrained')   
axs['oe'].set_xlim(0, MAX_ITER)
axs['oe'].set_ylim(9e-12, 9e0)
axs['oe'].set_yscale('log')
axs['oe'].set_xlabel(r'''$\mathrm{(a)}$''', fontsize=42)
# axs['oe'].set_ylabel(r'$|f({\mathbf x}^k)-f({\mathbf x}^\star)|$', fontsize=42)
axs['oe'].set_ylabel(r'objective error', fontsize=42)
axs['oe'].tick_params(axis='both', labelsize=42)
axs['oe'].yaxis.set_minor_locator(plt.LogLocator())
axs['oe'].spines.bottom.set_linewidth(0)
axs['oe'].spines.top.set_linewidth(0)
axs['oe'].spines.left.set_linewidth(0)
axs['oe'].spines.right.set_linewidth(0)
axs['oe'].grid(True, lw=4, color='1')


p4, = axs['oe'].plot(oe_logs['UDC_New1'][0],  
                     label='UDC-New1', 
                     c=c['UDC_New1'],
                     alpha=al['UDC_New1'],
                     lw=2, 
                     ls='-'
                     )
# p1, = axs['oe'].plot(oe_logs['UDC_PEXTRA'][0], 
#                      label='UDC-PEXTRA', 
#                      c=c['UDC_PEXTRA'],
#                      alpha=al['UDC_PEXTRA'],
#                      lw=2, 
#                      ls='-'
#                      )
# p2, = axs['oe'].plot(oe_logs['UDC_PGC'][0], 
#                      label='UDC-PGC', 
#                      c=c['UDC_PGC'],
#                      alpha=al['UDC_PGC'],
#                      lw=2, 
#                      ls='-'
#                      )
# p3, = axs['oe'].plot(oe_logs['UDC_DPGA'][0], 
#                      label='UDC-DPGA', 
#                      c=c['UDC_DPGA'],
#                      alpha=al['UDC_DPGA'],
#                      lw=2,
#                      ls='-'
#                      ) 
# p4, = axs['oe'].plot(oe_logs['UDC_DistADMM'][0],  
#                      label='UDC-dist.ADMM', 
#                      c=c['UDC_DistADMM'],
#                      alpha=al['UDC_DistADMM'],
#                      lw=2, 
#                      ls='-'
#                      )
p5, = axs['oe'].plot(oe_logs['ALT'][0], 
                     label='ALT', 
                     c=c['ALT'],
                     alpha=al['ALT'],
                     lw=2, 
                     ls='-'
                     )
# p6, = axs['oe'].plot(oe_logs['DPMM'], 
#                      label='DPMM', 
#                      c=c['DPMM'],
#                      alpha=al['DPMM'],
#                      lw=2, 
#                      ls='-.'
#                      )
# p7, = axs['oe'].plot(oe_logs['IPLUX'], 
#                      label='IPLUX', 
#                      c=c['IPLUX'],
#                      alpha=al['IPLUX'],
#                      lw=2, 
#                      ls='-.'
#                      )
# p8, = axs['oe'].plot(oe_logs['dual_subgradient'], 
#                      label='dual subgradient',  
#                      c=c['dual_subgradient'], 
#                      alpha=al['dual_subgradient'],
#                      lw=2,
#                      ls='-.'
#                      )


# ======================== constraint violation ===========================

# fig, ax = plt.subplots(figsize=(20, 12))   
axs['cv'].set_xlim(0, MAX_ITER)
axs['cv'].set_yscale('log')
axs['cv'].set_ylim(1e-7, 5e1)
axs['cv'].set_xlabel(r'''iteration $k$
                      
                      $\mathrm{(b)}$''', fontsize=42)
axs['cv'].set_ylabel(r'constraint violation', fontsize=42)
axs['cv'].tick_params(axis='both', labelsize=42)
axs['cv'].yaxis.set_minor_locator(plt.LogLocator())
axs['cv'].spines.bottom.set_linewidth(0)
axs['cv'].spines.top.set_linewidth(0)
axs['cv'].spines.left.set_linewidth(0)
axs['cv'].spines.right.set_linewidth(0)
axs['cv'].grid(True, lw=4, color='1')


p4, = axs['cv'].plot(cv_logs['UDC_New1'][0],  
                    #  label='UDC-New1', 
                     c=c['UDC_New1'],
                     alpha=al['UDC_New1'],
                     lw=2, 
                     ls='-'
                     )
# p1, = axs['cv'].plot(cv_logs['UDC_PEXTRA'][0], 
#                     #  label='UDC-PEXTRA', 
#                      c=c['UDC_PEXTRA'],
#                      alpha=al['UDC_PEXTRA'],
#                      lw=2, 
#                      ls='-'
#                      )
# p2, = axs['cv'].plot(cv_logs['UDC_PGC'][0], 
#                     #  label='UDC-PGC', 
#                      c=c['UDC_PGC'],
#                      alpha=al['UDC_PGC'],
#                      lw=2, 
#                      ls='-'
#                      )
# p3, = axs['cv'].plot(cv_logs['UDC_DPGA'][0], 
#                     #  label='UDC-DPGA', 
#                      c=c['UDC_DPGA'],
#                      alpha=al['UDC_DPGA'],
#                      lw=2,
#                      ls='-'
#                      ) 
# p4, = axs['cv'].plot(cv_logs['UDC_DistADMM'][0],  
#                     #  label='UDC-dist.ADMM', 
#                      c=c['UDC_DistADMM'],
#                      alpha=al['UDC_DistADMM'],
#                      lw=2, 
#                      ls='-'
#                      )
p5, = axs['cv'].plot(cv_logs['ALT'][0], 
                    #  label='ALT', 
                     c=c['ALT'],
                     alpha=al['ALT'],
                     lw=2, 
                     ls='-'
                     )
# p6, = axs['cv'].plot(cv_logs['DPMM'], 
#                     #  label='DPMM', 
#                      c=c['DPMM'],
#                      alpha=al['DPMM'],
#                      lw=2, 
#                      ls='-.'
#                      )
# p7, = axs['cv'].plot(cv_logs['IPLUX'], 
#                     #  label='IPLUX', 
#                      c=c['IPLUX'],
#                      alpha=al['IPLUX'],
#                      lw=2, 
#                      ls='-.'
#                      )
# p8, = axs['cv'].plot(cv_logs['dual_subgradient'], 
#                     #  label='dual subgradient',  
#                      c=c['dual_subgradient'], 
#                      alpha=al['dual_subgradient'],
#                      lw=2,
#                      ls='-.'
#                      )

first_legend = fig.legend(fontsize=40, loc='outside upper left', ncols=2)



# ================================= Proximal ====================================

fig.add_artist(first_legend)

alpha_list = [0, 0.1, 0.5]
# set linestyle
ls = {0:':', 0.1:'-', 0.5: '-'}
# set transparent
al = {0:1, 0.1:1, 0.5:0.5}
marker = {0:'*', 0.1:'', 0.5:''}
handles = [] # line handles, for legend


axs['oep'].set_xlim(0, MAX_ITER)
axs['oep'].set_yscale('log')
axs['oep'].set_ylim(1e-9, 9e0)
axs['oep'].set_xlabel(r'$\mathrm{(c)}$', fontsize=42)
# axs['oep'].set_ylabel(r'objective error', fontsize=42)
axs['oep'].spines.bottom.set_linewidth(0)
axs['oep'].spines.top.set_linewidth(0)
axs['oep'].spines.left.set_linewidth(0)
axs['oep'].spines.right.set_linewidth(0)
axs['oep'].grid(True, lw=4, color='1')

for alpha in alpha_list:
    pad = ''
    p0, = axs['oep'].plot(oe_logs['UDC_New1'][alpha],  
                     label=pad+f'$\\alpha={alpha}$', 
                     c=c['UDC_New1'],
                     alpha=al[alpha],
                     lw=2, 
                     ls=ls[alpha]
                     )
    # # p1, = axs['oep'].plot(oe_logs['UDC_PEXTRA'][alpha], 
    # #                         label=pad+f'$\\alpha={alpha}$', 
    # #                         c=c['UDC_PEXTRA'],
    # #                         alpha=al[alpha],
    # #                         lw=2, 
    # #                         # marker=marker[alpha],
    # #                         # markevery=50,
    # #                         # markerfacecolor='red',
    # #                         # fillstyle='full',
    # #                         # markersize=20,
    # #                         # markeredgecolor='red',
    # #                         # markeredgewidth=3,
    # #                         ls=ls[alpha]
    # #                         )
    # p2, = axs['oep'].plot(oe_logs['UDC_PGC'][alpha], 
    #                         label=pad+f'$\\alpha={alpha}$', 
    #                         c=c['UDC_PGC'],
    #                         alpha=al[alpha],
    #                         lw=2, 
    #                         ls=ls[alpha]
    #                         )
    # # p3, = axs['oep'].plot(oe_logs['UDC_DPGA'][alpha], 
    # #                         label=pad+f'$\\alpha={alpha}$', 
    # #                         c=c['UDC_DPGA'],
    # #                         alpha=al[alpha],
    # #                         lw=2, 
    # #                         ls=ls[alpha]
    # #                         )
    # p4, = axs['oep'].plot(oe_logs['UDC_DistADMM'][alpha], 
    #                         label=pad+f'$\\alpha={alpha}$', 
    #                         c=c['UDC_DistADMM'],
    #                         alpha=al[alpha],
    #                         lw=2, 
    #                         ls=ls[alpha]
    #                         )
    p5, = axs['oep'].plot(oe_logs['ALT'][alpha], 
                            label=pad+f'$\\alpha={alpha}$', 
                            c=c['ALT'],
                            alpha=al[alpha],
                            lw=2, 
                            ls=ls[alpha]
                            )
    # handles += [p0, p2, p4, p5]
    handles += [p0, p5]

axs['cvp'].set_xlim(0, MAX_ITER)
axs['cvp'].set_yscale('log')
axs['cvp'].set_ylim(1e-10, 5e1)
axs['cvp'].set_xlabel(r'''iteration $k$
                      
                      $\mathrm{(d)}$''', fontsize=42)
# axs['cvp'].set_ylabel(r'objective error', fontsize=42)
axs['cvp'].spines.bottom.set_linewidth(0)
axs['cvp'].spines.top.set_linewidth(0)
axs['cvp'].spines.left.set_linewidth(0)
axs['cvp'].spines.right.set_linewidth(0)
axs['cvp'].grid(True, lw=4, color='1')

for alpha in alpha_list:
    p0, = axs['cvp'].plot(cv_logs['UDC_New1'][alpha],  
                    #  label=f'$\\alpha={alpha}$', 
                     c=c['UDC_New1'],
                     alpha=al[alpha],
                     lw=2, 
                     ls=ls[alpha]
                     )
    # # p1, = axs['cvp'].plot(cv_logs['UDC_PEXTRA'][alpha], 
    # #                         # label='UDC-PEXTRA', 
    # #                         c=c['UDC_PEXTRA'],
    # #                         alpha=al[alpha],
    # #                         lw=2, 
    # #                         ls=ls[alpha]
    # #                         )
    # p1, = axs['cvp'].plot(cv_logs['UDC_PGC'][alpha], 
    #                         # label='UDC-PGC', 
    #                         c=c['UDC_PGC'],
    #                         alpha=al[alpha],
    #                         lw=2, 
    #                         ls=ls[alpha]
    #                         )
    # # p1, = axs['cvp'].plot(cv_logs['UDC_DPGA'][alpha], 
    # #                         # label='UDC-DPGA', 
    # #                         c=c['UDC_DPGA'],
    # #                         alpha=al[alpha],
    # #                         lw=2, 
    # #                         ls=ls[alpha]
    # #                         )
    # p1, = axs['cvp'].plot(cv_logs['UDC_DistADMM'][alpha], 
    #                         # label='UDC-dist.ADMM', 
    #                         c=c['UDC_DistADMM'],
    #                         alpha=al[alpha],
    #                         lw=2, 
    #                         ls=ls[alpha]
    #                         )
    p1, = axs['cvp'].plot(cv_logs['ALT'][alpha], 
                            # label='ALT', 
                            c=c['ALT'],
                            alpha=al[alpha],
                            lw=2, 
                            ls=ls[alpha]
                            )



fig.legend(handles=handles,fontsize=40, loc='outside upper right', ncols=3)
fig.savefig(f'compare_obj_err.pdf', bbox_inches='tight', transparent=False)

In [ ]:

# ============================= trace in x1 x1 plane =========================
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()     # Create a figure containing a single Axes.

delta = 0.05
x = np.arange(0.0, 1, delta)
y = np.arange(0.0, 1, delta)
X, Y = np.meshgrid(x, y)
F = X*alg.c[0] + Y*alg.c[1]
G = -alg.d[0]*np.log(1+X) - alg.d[1]*np.log(1+Y) + alg.b[0]*2/num_node

im = ax.imshow(F, interpolation='bilinear', origin='lower',
            cmap=cm.gray, extent=(0, 1, 0, 1))
CS_F = ax.contour(X, Y, F, 5)
ax.clabel(CS_F, fontsize=50)

# if there are only two nodes, we can polt the constraint
if num_node == 2:
    CS_G = ax.contour(X, Y, G, colors='k')
    ax.clabel(CS_G, fontsize=20)

x_log = np.array(alg.x_log)
x_avg_log = np.array(alg.x_avg_log)
ax.scatter(alg.x_star[0], alg.x_star[1], s=5000, c='r', marker='X', label='optimal')
ax.scatter(x_log[-1,0], x_log[-1,1], s=500, c='y', marker='^', label='last')
ax.scatter(x_log[:,0], x_log[:,1], label='iter', s=100, c='b', marker='^')  # Plot some data on the Axes.
ax.scatter(x_avg_log[:,0], x_avg_log[:,1], label='iter avg', s=100, marker='^')

ax.set_title('x position')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.legend()
fig.savefig('x.png')
plt.close()
